In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import scipy.io as sio
import time
import re
from pathlib import Path
import csv
import utils_oe

In [8]:
bhv_path = Path(
        "C:/Users/camil/Documents/int/data/openephys/2022-06-09_10-40-34/Record Node 102/experiment1/recording1/220609_TSCM_grid_Riesling.h5"
    ) 

In [12]:
bhv['Trial1'].keys()

<KeysViewHDF5 ['Trial', 'BlockCount', 'TrialWithinBlock', 'Block', 'Condition', 'TrialError', 'ReactionTime', 'AbsoluteTrialStartTime', 'TrialDateTime', 'BehavioralCodes', 'AnalogData', 'ObjectStatusRecord', 'RewardRecord', 'UserVars', 'VariableChanges', 'TaskObject', 'CycleRate', 'Ver']>

In [44]:
split_name = re.split(r"[/]", f['/ML/Trial1/TaskObject/Attribute/1'].name)

In [75]:
f['/ML/Trial1/TaskObject/Attribute/1/3']

<HDF5 dataset "3": shape (1, 1), type "<f8">

In [83]:
np.array(f['/ML/Trial1/TaskObject/Attribute/1/1']).item().decode("utf-8") 

'fix'

In [84]:
trials_keys =  list(bhv.keys())[1:-1]

In [122]:
re.split(r"[Trial]", trial)

['', '', '', '', '', '528']

In [98]:
int(re.split(r"[Trial]", trial)[-1])

528

In [142]:
def visitor_func(name, node):
    if isinstance(node, h5py.Dataset):
        # split node name
        split_name = re.split(r"[/]", node.name)
        if split_name[2] != "MLConfig" and split_name[2] != "TrialRecord":
            n_trial = int(re.split(r"[Trial]", split_name[2])[-1])

            node_name = split_name[-1]
            node_data = np.array(node)
            if len(split_name) >4 and split_name[4] == 'Attribute':
                node_name = split_name[4] + split_name[5] + split_name[6] 
                if node_name == 'Attribute11'or node_name == 'Attribute21':
                    node_data = np.array(node).item().decode("utf-8") 

            if len(split_name) >4 and split_name[4] == 'Position':
                node_name = split_name[4] + split_name[5] 

            
            bhv_res[n_trial-1][node_name] = node_data 

# create list of dicts, where each dict is a trial
bhv_res = []
for trial in trials_keys:
    bhv_res.append({'trial':re.split(r"[Trial]", trial)[-1]})

f = h5py.File(bhv_path, "r")["ML"]
f.visititems(visitor_func)


In [131]:
bhv_res[0].keys()

dict_keys(['trial', 'AbsoluteTrialStartTime', 'Btn1', 'Btn10', 'Btn2', 'Btn3', 'Btn4', 'Btn5', 'Btn6', 'Btn7', 'Btn8', 'Btn9', 'Eye', 'Eye2', 'EyeExtra', 'Gen1', 'Gen10', 'Gen11', 'Gen12', 'Gen13', 'Gen14', 'Gen15', 'Gen16', 'Gen2', 'Gen3', 'Gen4', 'Gen5', 'Gen6', 'Gen7', 'Gen8', 'Gen9', 'Joystick', 'Joystick2', 'KeyInput', 'Mouse', 'PhotoDiode', 'SampleInterval', 'Touch', 'CodeNumbers', 'CodeTimes', 'Block', 'BlockCount', 'Condition', 'CycleRate', 'Angle', 'BackgroundColor', 'Info', 'Position1', 'Position2', 'Position3', 'Position4', 'Scale', 'Status', 'Time', 'Zorder', 'ReactionTime', 'EndTimes', 'StartTimes', 'Attribute11', 'Attribute12', 'Attribute13', 'Attribute21', 'Attribute22', 'Attribute23', 'Attribute24', 'Attribute25', 'Attribute26', 'Code', 'FrameByFrameMovie', 'Size', 'Trial', 'TrialDateTime', 'TrialError', 'TrialWithinBlock', 'SkippedFrameTimeInfo', 'Dist', 'Eye2Offset', 'EyeOffset', 'Fix_FP_T_time', 'Fix_FP_post_T_time', 'Fix_FP_pre_T_time', 'Fixclose', 'Fixfar', 'ITI_va

In [143]:
bhv_res[0]['Attribute21']

'crc'

# Guilhem code

In [ ]:
dset3 = f1.create_dataset("FullWord", data=FullWord)
dset4 = f1.create_dataset("Strobes", data=Real_strobes.timestamp.array)

In [ ]:
# BHV files from ML using either h5 or mat format
#filename = "/home/gibos/disks/envau_H/data/openephys/210909_TSCM_5cj_5cl_Riesling.h5"
#filename = "/home/gibos/disks/envau_H/data/openephys/210910_TSCM_5cj_5cl_Riesling.mat"

Numtrial=BHV['TrialRecord']['CurrentTrialNumber'][0][0][0][0]
Trial='Trial'
BHVcodes = []
Errors   = []
scol= np.empty((Numtrial,1,))
scol[:]=np.nan
sor= np.empty((Numtrial,1,))
sor[:]=np.nan
Cond=np.empty((Numtrial,1,))
Cond[:]=np.nan
Block=np.empty((Numtrial,1,))
Block[:]=np.nan

for i in range(Numtrial):
    current_trial=Trial+str(i+1)
    tmp=list(np.concatenate(BHV[current_trial]['BehavioralCodes'][0][0][0][0][1][:]))
    BHVcodes.append(list(np.concatenate(BHV[current_trial]['BehavioralCodes'][0][0][0][0][1])))
    Cond[i]=BHV[current_trial]['Condition'][0][0][0][0]
    Block[i]=BHV[current_trial]['Block'][0][0][0][0]
    if (Cond[i][0]>=1) & (Cond[i][0]<=6):
        scol[i]=1
        sor[i]=1
    elif (Cond[i][0]>6) & (Cond[i][0]<=12):
        scol[i]=5
        sor[i]=1
    elif (Cond[i][0]>12) & (Cond[i][0]<=18):
        scol[i]=1
        sor[i]=5
    elif (Cond[i][0]>18) :
        scol[i]=5
        sor[i]=5
    #print(current_trial)
    #BHVCodes=BHV[current_trial]['BehavioralCodes'][0][0][0][0][1][:].shape
    Errors.append(BHV[current_trial]['TrialError'][0][0][0][0])
    
BHVcodes=np.concatenate(BHVcodes)
Errors=np.array(Errors, dtype='int')

In [ ]:
dset5 = f1.create_dataset("Error", data=Errors)
dset6= f1.create_dataset("Block", data=np.array(np.concatenate(Block), dtype='int'))

In [ ]:
## Final codes and timestamps

Codes=Real_strobes
#Codes=Codes.drop(Indices[np.where(Diff_TS<2)])

Codes.insert(5,'FullWord', FullWord, allow_duplicates=True)
Codes=Codes.drop(columns=['channel', 'processor_id', 'subprocessor_id', 'state'])
#Real_strobes.timestamp



In [ ]:
## Get spikes, need to work on integrating automated spike sorting: Spike interface

In [ ]:
## Start building trials For DCM Task

# 1 Start trial
# 2 End trial
# 3 []
# 4 Bar up (release)
# 7 Bar down (hold)
# 8 fixation occurs

# 9 Start trial
# 13 Frame skipped
# 14 Manual reward
# 18 End trial

# 23 Sample ON
# 24 Sample OFF
# 25 Test1 ON
# 26 Test1 OFF
# 27 Test2 ON
# 28 Test2 OFF
# 29 Test3 ON
# 30 Test3 OFF
# 31 Test4 ON
# 32 Test4 OFF
# 33 Test5 ON
# 34 Test5 OFF

# 35 Fixation spot ON
# 36 Fixation spot OFF

# 96 Reward given
# 97 broke fixation

######################################################
# code array: m*n, where m is the number of trials
# column 0: abslute timing of code 9
# column 1: abslute timing of code 7
# column 2: abslute timing of code 35  (FP ON)
# column 3: abslute timing of code 8  (fix ON)
# column 4: abslute timing of code 23  (Sample ON)
# column 5: Sample color
# column 6: Sample Orientation
# column 7: abslute timing of code 25  (Test1 ON)
# column 8: Test1 color
# column 9: Test1 orientation

# column 10: abslute timing of code 27  (Test2 ON)
# column 11: Test2 color
# column 12: Test2 orientation

# column 13: abslute timing of code 29  (Test3 ON)
# column 14: Test3 color
# column 15: Test3 orientation

# column 16: abslute timing of code 31  (Test4 ON)
# column 17: Test4 color
# column 18: Test4 orientation

# column 19: abslute timing of code 33  (Test5 ON)
# column 20: Test5 color
# column 21: Test5 orientation

# column 22: abslute timing of code 7  (bar release)
# column 23: abslute timing of code 96  (Reward)
# column 24: abslute timing of code 97  (break fix)
# column 25: abslute timing of code 18 (end trial)
# column 26: Error type  (0=correct, 3=BF, 6=FA, 8=Miss)
######################################################
Numtrial_DCM=np.where(Block==1)[0].shape[0]
trial_DCM=np.where(Block==1)[0]
if Numtrial_DCM>0:
    C = np.empty((Numtrial_DCM,27,))
    C[:]=np.nan
    C[:, -1]=np.array(Errors[trial_DCM])
    SessionCodes=pd.DataFrame(C, columns=['Start','Touch', 'Fix spot on', 'Fixation', 'Sample', 'sample color', 'sample orientation', 'Test 1', 'Test 1 color', 'Test 1 orientation', 'Test 2', 'Test 2 color', 'Test 2 orientation', 
                                          'Test 3', 'Test 3 color', 'Test 3 orientation', 'Test 4', 'Test 4 color', 'Test 4 orientation', 'Test 5', 'Test 5 color', 'Test 5 orientation', 'bar release', 'Reward', 'Break fix', 'End', 'Error type'])

    Correct_trials = np.where(np.array(Errors[trial_DCM])==0)
    Miss_trials    = np.where(np.array(Errors[trial_DCM])==8)
    FA_trials      = np.where(np.array(Errors[trial_DCM])==6)
    BF_trials      = np.where(np.array(Errors[trial_DCM])==3)

    SessionCodes['Start']      = Codes.timestamp[Codes.FullWord==9].values
    SessionCodes['End']  = Codes.timestamp[Codes.FullWord==18].values
    #SessionCodes['Touch']      = Codes.timestamp[Codes.FullWord==7].values
    #SessionCodes['Fix spot on']= Codes.timestamp[Codes.FullWord==35].values
    #SessionCodes['sample']     = Codes.timestamp[Codes.FullWord==23].values
    #SessionCodes['Test1']      = Codes.timestamp[Codes.FullWord==23].values

    for i in range(Numtrial):
        Tstart = SessionCodes.Start[i]
        Tend   = SessionCodes.End[i]
        current_trial=Codes.loc[(Codes['timestamp']>=Tstart) & (Codes['timestamp']<=Tend)]
        ct=Trial+str(i+1)

        for count, W in enumerate(current_trial.index):
            if current_trial.FullWord[W]==7:
                SessionCodes.loc[i, ['Touch']]=current_trial.timestamp[W]
            elif current_trial.FullWord[W]==35:
                SessionCodes.loc[i, ['Fix spot on']]=current_trial.timestamp[W]
            elif current_trial.FullWord[W]==8:
                SessionCodes.loc[i, ['Fixation']]=current_trial.timestamp[W]
            elif current_trial.FullWord[W]==23:
                SessionCodes.loc[i, ['Sample']]=current_trial.timestamp[W]
                SessionCodes.loc[i, ['sample color']]=scol[i]
                SessionCodes.loc[i, ['sample orientation']]=sor[i]
            elif current_trial.FullWord[W]==25:
                SessionCodes.loc[i, ['Test 1']]=current_trial.timestamp[W]
                SessionCodes.loc[i, ['Test 1 orientation']] =    int(BHV[ct]['UserVars'][0][0][0][0][0][0][7])
                SessionCodes.loc[i, ['Test 1 color']]       =    int(BHV[ct]['UserVars'][0][0][0][0][0][0][11])
            elif current_trial.FullWord[W]==27:
                SessionCodes.loc[i, ['Test 2']]=current_trial.timestamp[W]  
                SessionCodes.loc[i, ['Test 2 orientation']] =   int(BHV[ct]['UserVars'][0][0][0][0][1][0][7])
                SessionCodes.loc[i, ['Test 2 color']]       =   int(BHV[ct]['UserVars'][0][0][0][0][1][0][11])
            elif current_trial.FullWord[W]==29:
                SessionCodes.loc[i, ['Test 3']]=current_trial.timestamp[W]  
                SessionCodes.loc[i, ['Test 3 orientation']] =   int(BHV[ct]['UserVars'][0][0][0][0][2][0][7])
                SessionCodes.loc[i, ['Test 3 color']]       =   int(BHV[ct]['UserVars'][0][0][0][0][2][0][11])
            elif current_trial.FullWord[W]==31:
                SessionCodes.loc[i, ['Test 4']]=current_trial.timestamp[W]   
                SessionCodes.loc[i, ['Test 4 orientation']] =   int(BHV[ct]['UserVars'][0][0][0][0][3][0][7])
                SessionCodes.loc[i, ['Test 4 color']]       =   int(BHV[ct]['UserVars'][0][0][0][0][3][0][11])
            elif current_trial.FullWord[W]==33:
                SessionCodes.loc[i, ['Test 5']]=current_trial.timestamp[W]  
                SessionCodes.loc[i, ['Test 5 orientation']] =   int(BHV[ct]['UserVars'][0][0][0][0][4][0][7])
                SessionCodes.loc[i, ['Test 5 color']]       =   int(BHV[ct]['UserVars'][0][0][0][0][4][0][11])
            elif current_trial.FullWord[W]==4:
                SessionCodes.loc[i, ['bar release']]=current_trial.timestamp[W]    
            elif current_trial.FullWord[W]==96:
                SessionCodes.loc[i, ['Reward']]=current_trial.timestamp[W]    
            elif current_trial.FullWord[W]==97:
                SessionCodes.loc[i, ['Break fix']]=current_trial.timestamp[W]    

            
            
    SessionCodes

In [ ]:
## building trials For Mem saccade grids

# 8 acquire fix
# 10 Target Back ON

# 9 Start trial
# 13 Frame skipped
# 14 Manual reward
# 18 End trial

# 35 Fix ON 
# 36 Fix OFF
# 37 Target ON
# 38 Target OFF

# 40 Target back OFF

# 35 Fixation spot ON
# 36 Fixation spot OFF

# 50 Break Fixation 1
# 51 Break Fixation 2
# 53 Break Fixation 3

# 56 Reward given
# Large grid
# 80  X:13   Y:-13
# 81  X:9    Y:-13
# 82  X:5    Y:-13
# 83  X:0    Y:-13
# 84  X:-5   Y:-13
# 85  X:-9   Y:-13
# 86  X:-13  Y:-13

# 87  X:13   Y:-9
# 88  X:9    Y:-9
# 89  X:5    Y:-9
# 90  X:0    Y:-9
# 91  X:-5   Y:-9
# 92  X:-9   Y:-9
# 93  X:-13  Y:-9

# 94  X:13   Y:-5
# 95  X:9    Y:-5
# 96  X:5    Y:-5
# 97  X:0    Y:-5
# 98  X:-5   Y:-5
# 99  X:-9   Y:-5
# 100 X:-13  Y:-5

# 101 X:13    Y:0
# 102 X:9     Y:0
# 103 X:5     Y:0
# 104 X:-5    Y:0
# 105 X:-9    Y:0
# 106 X:-13   Y:0

# 107 X:13    Y:5
# 108 X:9     Y:5
# 109 X:5     Y:5
# 110 X:0     Y:5
# 111 X:-5    Y:5
# 112 X:-9    Y:5
# 113 X:-13   Y:5
# 113 X:-13   Y:5

# 114 X:13    Y:9
# 115 X:9     Y:9
# 116 X:5     Y:9
# 117 X:0     Y:9
# 118 X:-5    Y:9
# 119 X:-9    Y:9
# 120 X:-13   Y:9

# 121 X:13    Y:13
# 122 X:9     Y:13
# 123 X:5     Y:13
# 124 X:0     Y:13
# 125 X:-5    Y:13
# 126 X:-9    Y:13
# 127 X:-13   Y:13



# Small grid 

# 87  X:13   Y:-9
# 88  X:9    Y:-9
# 89  X:5    Y:-9
# 90  X:0    Y:-9
# 91  X:-5   Y:-9
# 92  X:-9   Y:-9
# 93  X:-13  Y:-9

# 94  X:13   Y:-5
# 95  X:9    Y:-5
# 96  X:5    Y:-5
# 97  X:0    Y:-5
# 98  X:-5   Y:-5
# 99  X:-9   Y:-5
# 100 X:-13  Y:-5

# 101 X:13    Y:0
# 102 X:9     Y:0
# 103 X:5     Y:0
# 104 X:-5    Y:0
# 105 X:-9    Y:0
# 106 X:-13   Y:0

# 107 X:13    Y:5
# 108 X:9     Y:5
# 109 X:5     Y:5
# 110 X:0     Y:5
# 111 X:-5    Y:5
# 112 X:-9    Y:5
# 113 X:-13   Y:5
# 113 X:-13   Y:5

# 114 X:13    Y:9
# 115 X:9     Y:9
# 116 X:5     Y:9
# 117 X:0     Y:9
# 118 X:-5    Y:9
# 119 X:-9    Y:9
# 120 X:-13   Y:9


######################################################
# code array: m*n, where m is the number of trials
# column 0: abslute timing of code 9   (Trial start)
# column 1: abslute timing of code 35  (FP ON)
# column 2: abslute timing of code 36  (FP OFF)
# column 3: abslute timing of code 37  (Target ON)
# column 4: abslute timing of code 38  (Target OFF)
# column 5: abslute timing of code 38  (Target back ON)
# column 6: abslute timing of code 40  (Target back OFF)

# column 7: abslute timing of code 8  (acquire fix)
# column 8: abslute timing of code 50 (break fix 1)
# column 9: abslute timing of code 51  (break fix 2)
# column 10: abslute timing of code 52  (break fix 3)

# column 11: abslute timing of code 56  (Reward)
# column 12: Target X  (Pos)
# column 13: Target Y  (Pos)

# column 14: abslute timing of code 18 (end trial)
######################################################

In [ ]:
Numtrial_MSgrid= np.where(Block==2)[0].shape[0]
trial_MSgrid   = np.array(np.where(Block==2)[0], int)
X=[13, 9, 5, 0, -5, -9, -13, 13, 9, 5, 0, -5, -9, -13, 13, 9, 5, 0, -5, -9, -13, 13, 9, 5, -5, -9, -13, 
   13, 9, 5, 0, -5, -9, -13, 13, 9, 5, 0, -5, -9, -13, 13, 9, 5, 0, -5, -9, -13]
Y=np.zeros(48)
Y[0:7]=-13
Y[7:14]=-9
Y[14:21]=-5
Y[21:27]=0
Y[27:34]=5
Y[34:41]=9
Y[41:48]=13
if Numtrial_MSgrid>0:
    C = np.empty((Numtrial_MSgrid,16,))
    C[:]=np.nan
    C[:, -1]=np.array(Errors[trial_MSgrid])
    SessionCodes=pd.DataFrame(C, columns=['Start','Fix spot on', 'Fix spot off', 'Target On', 'Target Off', 'Target back On',  'Target back Off',
                                          'acquire fix', 'Break fix 1', 'Break fix 2', 'Break fix 3', 'Reward', 'Pos X', 'Pos Y', 'Error','End'])

    Correct_trials = np.where(np.array(Errors[trial_MSgrid])==0)
    

    SessionCodes['Start']      = Codes.timestamp[Codes.FullWord==9].values
    SessionCodes['End']        = Codes.timestamp[Codes.FullWord==18].values
    #SessionCodes['Touch']      = Codes.timestamp[Codes.FullWord==7].values
    #SessionCodes['Fix spot on']= Codes.timestamp[Codes.FullWord==35].values
    #SessionCodes['sample']     = Codes.timestamp[Codes.FullWord==23].values
    #SessionCodes['Test1']      = Codes.timestamp[Codes.FullWord==23].values
    SessionCodes['Error'] =Errors[trial_MSgrid]
    for i in range(Numtrial):
        Tstart = SessionCodes.Start[i]
        Tend   = SessionCodes.End[i]
        current_trial=Codes.loc[(Codes['timestamp']>=Tstart) & (Codes['timestamp']<=Tend)]
        ct=Trial+str(i+1)

        for count, W in enumerate(current_trial.index):
            if current_trial.FullWord[W]==35:
                SessionCodes.loc[i, ['Fix spot on']]=current_trial.timestamp[W]
            elif current_trial.FullWord[W]==8:
                SessionCodes.loc[i, ['acquire fix']]=current_trial.timestamp[W]
            elif current_trial.FullWord[W]==36:
                SessionCodes.loc[i, ['Fix spot off']]=current_trial.timestamp[W]
            elif current_trial.FullWord[W]==37:
                SessionCodes.loc[i, ['Target On']]=current_trial.timestamp[W]
            elif current_trial.FullWord[W]==38:
                SessionCodes.loc[i, ['Target Off']]=current_trial.timestamp[W]  
            elif current_trial.FullWord[W]==10:
                SessionCodes.loc[i, ['Target back On']]=current_trial.timestamp[W]  
            elif current_trial.FullWord[W]==40:
                SessionCodes.loc[i, ['Target back Off']]=current_trial.timestamp[W]  
            elif current_trial.FullWord[W]==50:
                SessionCodes.loc[i, ['Break fix 1']]=current_trial.timestamp[W]   
            elif current_trial.FullWord[W]==51:
                SessionCodes.loc[i, ['Break fix 2']]=current_trial.timestamp[W]  
            elif current_trial.FullWord[W]==53:
                SessionCodes.loc[i, ['Break fix 3']]=current_trial.timestamp[W]    
            elif current_trial.FullWord[W]==56:
                SessionCodes.loc[i, ['Reward']]=current_trial.timestamp[W]       
            elif current_trial.FullWord[W]>=80:
                SessionCodes.loc[i, ['Pos X']]=int(X[current_trial.FullWord[W]-80])
                SessionCodes.loc[i, ['Pos Y']]=int(Y[current_trial.FullWord[W]-80])
    

SessionCodes['Pos X']=SessionCodes['Pos X'].astype("Int64")
SessionCodes['Pos Y']=SessionCodes['Pos Y'].astype("Int64")

In [ ]:
recording_num

In [ ]:
Sp_filename = directory + 'Record Node 102/experiment1/recording' + str(recording_num+1) + '/continuous/Rhythm_FPGA-100.0/herdingspikes_output/phy_folder_for_HS/spike_times.npy'
spiketimes = [np.load(Sp_filename)]
spiketimes

In [ ]:

path=Sp_filename[0:Sp_filename.find('spike_times')]

date=path[path.find('2022'):path.find('2022')+10]

base_path=path[0:path.find('Rhythm_FPGA-100.0')+18]

timestamps=np.load(base_path+'timestamps.npy')

spiketimes = np.load(Sp_filename)
spikesamples=timestamps[spiketimes]
spiketimestamps=np.array(np.concatenate(np.round(timestamps[spiketimes]/30)), 'int') #put spikes in ms
spiketimes_clusters_id = np.load(path+'spike_clusters.npy')


In [ ]:
timestamps

In [ ]:
dset7 = f1.create_dataset("SpikeTS", data=spiketimestamps)
dset8 = f1.create_dataset("SpikeID", data=spiketimes_clusters_id)


In [ ]:
spike_cluster_table = []
for i, spike in enumerate(spiketimes):
    spike_cluster_table.append((spike[0], spiketimes_clusters_id[i]))
#for count, i in enumerate(np.unique(spiketimes_clusters_id)):
#    print([count, i])


In [ ]:
dset9 = f1.create_dataset("spike_cluster_table", data=spiketimes_clusters_id)

In [ ]:
good_clusters=[]
mua_clusters=[]

with open(path+'/cluster_info.tsv', 'r') as csvFile:
    
    reader = csv.reader(csvFile)
    i=0
    for row in reader:
        
        row_values = row[0].split('\t')
        cluster_id, channel, group = row_values[0], row_values[2], row_values[6]
        depth, n_spikes = row_values[4], row_values[7]
        
        if i>0 and group=='good':
            good_clusters.append([int(cluster_id), int(channel), float(depth), int(n_spikes)])   
        elif i>0 and group=='mua':
            mua_clusters.append([int(cluster_id), int(channel), float(depth), int(n_spikes)])   
        i=i+1#print(np.array(cluster_id)[1:, :])

In [ ]:
dset10=f1.create_dataset("good_clusters_info", data=good_clusters)
dset11=f1.create_dataset("mua_clusters_info", data=mua_clusters)

In [ ]:
Correct_trials=SessionCodes[SessionCodes['Error']==0].index
XY=[-13, -9, -5, 0, 5, 9, 13]

Pos_correct={}
LFP_correct={}

for count, i in enumerate(np.array(good_clusters)[:,0]):
    print(count)
    locals()['cell_{0}'.format(count)]={}
    
    locals()['cell_{0}'.format(count)]["SpikeTimes"]=spiketimestamps[spiketimes_clusters_id==int(i)]
    locals()['cell_{0}'.format(count)]["celldepth"]=int(good_clusters[count][2])
    locals()['cell_{0}'.format(count)]["Target On"]={}
    locals()['cell_{0}'.format(count)]["FP Off"]={}
    locals()['cell_{0}'.format(count)]["Target On"]["correct"]={}
    locals()['cell_{0}'.format(count)]["FP Off"]["correct"]={}

    T=spikesamples[spiketimes_clusters_id==int(i)]

    for j in range(7):
 
        locals()['cell_{0}'.format(count)]["Target On"]["correct"][j]={}
        locals()['cell_{0}'.format(count)]["FP Off"]["correct"][j]={}
        Pos_correct[j]={}
        
        for k in range(7):
            
            if XY[j]==0 and XY[k]==0:
                Pos_correct[j][k]='NaN' 
                Pass=True
            else:  
                Pos_correct[j][k]=SessionCodes[(SessionCodes['Error']==0) & (SessionCodes['Pos X']==XY[k]) & (SessionCodes['Pos Y']==XY[j])].index
                Pass=False
            if Pass==False:
                locals()['cell_{0}'.format(count)]["Target On"]["correct"][j][k]=np.zeros((Pos_correct[j][k].shape[0], 1901))
                locals()['cell_{0}'.format(count)]["FP Off"]["correct"][j][k]=np.zeros((Pos_correct[j][k].shape[0], 1901))
                for count2, Tr in enumerate(Pos_correct[j][k]):
                    T_ON_TS=SessionCodes['Target On'][Tr]
                    FP_OFF_TS=SessionCodes['Fix spot off'][Tr]
                    D=T[np.where((T>=T_ON_TS-700*30) & (T<=T_ON_TS+1200*30))]-T_ON_TS
                    D2=np.array(np.round(D/30)+700, dtype='int')
                    locals()['cell_{0}'.format(count)]["Target On"]["correct"][j][k][count2, D2]=1
                    
                    D=T[np.where((T>=FP_OFF_TS-1200*30) & (T<=FP_OFF_TS+700*30))]-FP_OFF_TS
                    D2=np.array(np.round(D/30)+1200, dtype='int')
                    locals()['cell_{0}'.format(count)]["FP Off"]["correct"][j][k][count2, D2]=1
                    
                    
    name=date+'-cell-{0}'.format(count)                
    filehandler = open(name, 'w') 
    
    #f1 = h5py.File(name, "w")
    #pickle.dump(locals()['cell_{0}'.format(count)], filehandler)
    #f1.create_dataset("default", data=locals()['cell_{0}'.format(count)])

In [ ]:

for count, i in enumerate(np.array(mua_clusters)[:,0]):
    locals()['MUA_{0}'.format(count)]={}
    locals()['MUA_{0}'.format(count)]["SpikeTimes"]=spiketimestamps[spiketimes_clusters_id==int(i)]
    locals()['MUA_{0}'.format(count)]["celldepth"]=int(mua_clusters[count][2])
    locals()['MUA_{0}'.format(count)]["Target On"]={}
    locals()['MUA_{0}'.format(count)]["FP Off"]={}
    locals()['MUA_{0}'.format(count)]["Target On"]["correct"]={}
    locals()['MUA_{0}'.format(count)]["FP Off"]["correct"]={}

    T=spikesamples[spiketimes_clusters_id==int(i)]

    for j in range(7):
 
        locals()['MUA_{0}'.format(count)]["Target On"]["correct"][j]={}
        locals()['MUA_{0}'.format(count)]["FP Off"]["correct"][j]={}
        Pos_correct[j]={}
        
        for k in range(7):
            
            if XY[j]==0 and XY[k]==0:
                Pos_correct[j][k]='NaN' 
                Pass=True
            else:  
                Pos_correct[j][k]=SessionCodes[(SessionCodes['Error']==0) & (SessionCodes['Pos X']==XY[k]) & (SessionCodes['Pos Y']==XY[j])].index
                Pass=False
            if Pass==False:
                
                locals()['MUA_{0}'.format(count)]["Target On"]["correct"][j][k]=np.zeros((Pos_correct[j][k].shape[0], 1901))
                locals()['MUA_{0}'.format(count)]["FP Off"]["correct"][j][k]=np.zeros((Pos_correct[j][k].shape[0], 1901))
                
                for count2, Tr in enumerate(Pos_correct[j][k]):
                    T_ON_TS=SessionCodes['Target On'][Tr]
                    FP_OFF_TS=SessionCodes['Fix spot off'][Tr]
                    
                    D=T[np.where((T>=T_ON_TS-700*30) & (T<=T_ON_TS+1200*30))]-T_ON_TS
                    D2=np.array(np.round(D/30)+700, dtype='int')
                    locals()['MUA_{0}'.format(count)]["Target On"]["correct"][j][k][count2, D2]=1
                                        
                    D=T[np.where((T>=FP_OFF_TS-1200*30) & (T<=FP_OFF_TS+700*30))]-FP_OFF_TS
                    D2=np.array(np.round(D/30)+1200, dtype='int')
                    locals()['MUA_{0}'.format(count)]["FP Off"]["correct"][j][k][count2, D2]=1

In [ ]:
# plot raster
plotraster=True
if plotraster==True:
    time_fp=np.arange(-700, 1201, 1, dtype='int')
    time_sacc=np.arange(-1200, 701, 1, dtype='int')
    for cell in range(np.array(good_clusters).shape[0]):
        active_cell=locals()['cell_{0}'.format(cell)]

        fig = plt.figure()
        plt.figure(figsize=(20, 20))
        l=1
        for i in range(7):
            for j in range(7):

                plt.subplot(7,7,l)
                if j==3 and i==3:
                    True
                else:
                    for T in range(active_cell['Target On']['correct'][i][j].shape[0]):
                        if time_fp[active_cell['Target On']['correct'][i][j][T,:]==1].shape[0]>0:                
                            plt.plot(time_fp[active_cell['Target On']['correct'][i][j][T,:]==1], np.zeros(int(np.sum(active_cell['Target On']['correct'][i][j][T,:])))+T,'|k')
                l=l+1   

        plt.savefig('C://Users//ibos.g//Documents//data//Taskswitching//figures//cell_'+str(cell)+'.jpg')                    
        plt.show()   
        plt.close
        fig = plt.figure()
        plt.figure(figsize=(20, 20))
        l=1
        for i in range(7):
            for j in range(7):

                plt.subplot(7,7,l)
                if j==3 and i==3:
                    True
                else:
                    for T in range(active_cell['FP Off']['correct'][i][j].shape[0]):
                        if time_sacc[active_cell['FP Off']['correct'][i][j][T,:]==1].shape[0]>0:                
                            plt.plot(time_sacc[active_cell['FP Off']['correct'][i][j][T,:]==1], np.zeros(int(np.sum(active_cell['FP Off']['correct'][i][j][T,:])))+T,'|k')
                l=l+1   

        plt.savefig('C://Users//ibos.g//Documents//data//Taskswitching//figures//cell_'+str(cell)+'sacc.jpg')                    
        plt.show()   
        plt.close

In [ ]:
if plotraster==True:
    for cell in range(np.array(mua_clusters).shape[0]):
        active_cell=locals()['MUA_{0}'.format(cell)]

        fig = plt.figure()
        plt.figure(figsize=(20, 20))
        l=1
        for i in range(7):
            for j in range(7):

                plt.subplot(7,7,l)
                if j==3 and i==3:
                    True
                else:
                    for T in range(active_cell['Target On']['correct'][i][j].shape[0]):
                        if time_fp[active_cell['Target On']['correct'][i][j][T,:]==1].shape[0]>0:                
                            plt.plot(time_fp[active_cell['Target On']['correct'][i][j][T,:]==1], np.zeros(int(np.sum(active_cell['Target On']['correct'][i][j][T,:])))+T,'|k')
                l=l+1   

        plt.savefig('C://Users//ibos.g//Documents//data//Taskswitching//figures//mua_'+str(cell)+'.jpg')                    
        plt.show()   

        fig = plt.figure()
        plt.figure(figsize=(20, 20))
        l=1
        for i in range(7):
            for j in range(7):

                plt.subplot(7,7,l)
                if j==3 and i==3:
                    True
                else:
                    for T in range(active_cell['FP Off']['correct'][i][j].shape[0]):
                        if time_sacc[active_cell['FP Off']['correct'][i][j][T,:]==1].shape[0]>0:                
                            plt.plot(time_sacc[active_cell['FP Off']['correct'][i][j][T,:]==1], np.zeros(int(np.sum(active_cell['FP Off']['correct'][i][j][T,:])))+T,'|k')
                l=l+1   

    plt.savefig('C://Users//ibos.g//Documents//data//Taskswitching//figures//mua_'+str(cell)+'sacc.jpg')                    
    plt.show()   

In [ ]:

Pos_correct={}
T=Timestamps

LFP_correct={}
LFP_correct["Target On"]={}
LFP_correct["FP Off"]={}
j=1
LFP_correct["Target On"][j]={}
LFP_correct["FP Off"][j]={}
Pos_correct[j]={}
k=1
Pos_correct[j][k]=SessionCodes[(SessionCodes['Error']==0) & (SessionCodes['Pos X']==XY[k]) & (SessionCodes['Pos Y']==XY[j])].index
LFP_correct["Target On"][j][k]=np.zeros((32, Pos_correct[j][k].shape[0], 1900))
count=0
Tr=Pos_correct[j][k][0]
T_ON_TS=SessionCodes['Target On'][Tr]
D=LFP_fd[np.where((T>=T_ON_TS-700*30) & (T<T_ON_TS+1200*30))[0],:32]
LFP_correct['Target On'][j][k][:, count,:]=np.rot90(D)

In [ ]:
LFP_correct['Target On'][j][k][:, count,:].shape

In [ ]:
Correct_trials=SessionCodes[SessionCodes['Error']==0].index


XY=[-13, -9, -5, 0, 5, 9, 13]


Pos_correct={}
T=Timestamps

LFP_correct={}
LFP_correct["Target On"]={}
LFP_correct["FP Off"]={}

for j in range(7):

    LFP_correct["Target On"][j]={}
    LFP_correct["FP Off"][j]={}
    Pos_correct[j]={}
    for k in range(7):     
        if XY[j]==0 and XY[k]==0:
            Pos_correct[j][k]='NaN'            
        else:
            Pos_correct[j][k]=SessionCodes[(SessionCodes['Error']==0) & (SessionCodes['Pos X']==XY[k]) & (SessionCodes['Pos Y']==XY[j])].index

            LFP_correct["Target On"][j][k]=np.zeros((32, Pos_correct[j][k].shape[0], 1900))
            #LFP_correct["FP Off"][j][k]={}


            for count, Tr in enumerate(Pos_correct[j][k]):
                T_ON_TS=SessionCodes['Target On'][Tr]
                
                D=LFP_fd[np.where((T>=T_ON_TS-700*30) & (T<T_ON_TS+1200*30))[0],:32]
                LFP_correct['Target On'][j][k][:, count,:]=np.rot90(D)
            
            
    #locals()['LFP_{0}'.format(Electrode)]=LFP_correct
    

In [ ]:
plt.figure(figsize=(20, 20))
l=1
time=np.arange(1901)-700
for i in range(7):
    for j in range(7):
        
        plt.subplot(7,7,l)
        if j==3 and i==3:
            True
        else:
            plt.plot(time_fp[:-1], np.mean(LFP_correct['Target On'][i][j][0,:,:],0))
            plt.plot([0, 0], [-300, 300],'r')
            
        l=l+1
        

In [ ]:
l=1
for i in range(7):
    for k in range(7):
        d2x=[]

        for n in range(32):
#    plt.subplot(1,2,1)
#    plt.plot(time_fp[:-1], np.mean(LFP_correct['Target On'][3][5][n,:,:],0)+n*1000)
            
            d1x=np.diff(np.mean(LFP_correct['Target On'][i][k][n,:,:],0),1)
            d2x.append(np.diff(d1x,1))  
    
            
        plt.imshow(d2x, aspect=10)
        l=l+1

In [ ]:
d1x.shape

In [ ]:
d2x+n*1000

In [ ]:
np.mean(LFP_correct['Target On'][j][i],0).shape
time.shape

In [ ]:
plt.plot(np.mean(LFP_correct['Target On'][3][5][n,:,:],0)-np.mean(LFP_correct['Target On'][3][5][n-1,:,:],0))